In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from beepy import beep
import torch.jit
from torch.optim.lr_scheduler import ExponentialLR
import time

In [ ]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(device)
x = torch.ones(1, device = device)
print(x)

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

In [ ]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_train)

In [ ]:
def show_image(image, label):
    assert(image.shape[0] == (3 * 32 * 32))
    red_channel = image[:1024].reshape(32, 32)
    green_channel = image[1024:2048].reshape(32, 32)
    blue_channel = image[2048:].reshape(32, 32)
    print(f"label: {label}")
    plt.figure(figsize=(0.75,0.5))
    rgb_image = np.stack([red_channel, green_channel, blue_channel], axis=2)
    plt.imshow(rgb_image)
    plt.axis('off')  # Turn off axis labels
    plt.show()

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        
        # Convolutional layers
        # VGG 1
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu0 = nn.ReLU()
        
        self.conv1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # VGG 2
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # VGG 3
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()
        
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Dropout layers
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 256)  # 64 channels, 4x4 image size after pooling
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        # VGG 1
        x = self.relu0(self.conv0(x))
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.dropout1(x)
        # VGG 2
        x = self.relu2(self.conv2(x))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.dropout2(x)
        # VGG 3
        x = self.relu4(self.conv4(x))
        x = self.pool5(self.relu5(self.conv5(x)))
        
        # Flatten the tensor for fully connected layers
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the SimpleCNN model
model = SimpleCNN(num_classes=10)
print(f"""No of threads: {torch. get_num_threads()}""")

def custom_weight_init(module):
    if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
        # Initialize weights using your custom logic
        nn.init.xavier_normal_(module.weight)
        
model.apply(custom_weight_init)

In [ ]:
def plot_loss(loss : list, epochs : list):
     plt.title('Cross Entropy Loss')
     plt.plot(epochs, loss, color='blue', label='train')

In [ ]:
state_dict_epoch = {
    "epoch": 0,
    "loss": 0,
    "state_dict":{}
}

loss_list = []
accuracy_list = []
num_epochs = 100
epochs = range(num_epochs)
#reset the model
model = SimpleCNN(num_classes=10)
model.apply(custom_weight_init)

In [ ]:
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.001)
scheduler = ExponentialLR(optimizer, gamma=0.9)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


start_time = time.time()

for epoch in epochs:
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        #print(labels.shape)
        optimizer.zero_grad()
        outputs = model(images)
        #print(outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    loss_list.append(loss.item())
        
    if(epoch == 0):   
        state_dict_epoch["epoch"] = epoch
        state_dict_epoch["loss"] = loss.item()
        state_dict_epoch["state_dict"] = model.state_dict()
    if(epoch > 1 and loss.item() < state_dict_epoch["loss"]):
        state_dict_epoch["epoch"] = epoch
        state_dict_epoch["loss"] = loss.item()
        state_dict_epoch["state_dict"] = model.state_dict()
    
    #scheduler.step()
    if((epoch +1)%(num_epochs / 5) == 0 or epoch == 0):    
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {loss.item():.4f}")
        
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

print(f"""Minimum loss has happened at epoch number {state_dict_epoch["epoch"]}""")
for _ in range(1):
    beep(sound = "success")
    
plot_loss(loss_list, epochs)

In [ ]:
torch.save(state_dict_epoch["state_dict"], "/Users/mohammadrezabeygifard/Desktop/practice/MachineLearning/CIFAR_10/model/model.pt")

In [ ]:
model.load_state_dict(torch.load("/Users/mohammadrezabeygifard/Desktop/practice/MachineLearning/CIFAR_10/model/model.pt"))
model.eval()

In [ ]:
batch_size = 1
validation_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.to(torch.device("cpu"))
model.eval()
correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for images, labels in validation_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        current_accuracy = 100 * correct / total
        accuracy_list.append(current_accuracy)
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")